# Connections between MKs 
### **MK = Member of the knesset  (חבר כנסת)
Based on transcripts of the knesset committees.<br/>
The work was done in the 'public knowledge workshop' hackathon and won 3rd place prize.

In [1]:
import pandas as pd
import networkx as nx

rows = []
protocol_rows = []

## Collecting committees texts and analyzing it.
We used a library called fataflows since it is the one used by the 'public knowledge workshop' we worked with.<br/>
The data download is done in parts - each committee text is divided to parts and processed separately.<br/>
Downloading each knesset data and analyzing it took arround 6 hourd per knesset.<br/>
This is why we kept a cache of the downloaded data and saved files of the analyzed data for each knesset.

## Constants

In [4]:
# Limit processing of protocol parts for development, -1 means no limit.
PROCESS_PARTS_LIMIT = 1

# Enable caching of protocol parts data (not efficient, should only be used for local development with sensible PROCESS_PARTS_LIMIT)
PROCESS_PARTS_CACHE = True

# Filter the meetings to be processed, these kwargs are passed along to DataFlows filter_rows processor for meetings resource
MEETINGS_FILTER_ROWS_KWARGS = {'equals': [{'KnessetNum': 20}]}

# Don'e use local data - loads everything from knesset data remote storage
# When set to False - also enables caching, so you won't download from remote storage on 2nd run.
USE_DATA = False

## Load source data

In [37]:
from dataflows import filter_rows, cache
from datapackage_pipelines_knesset.common_flow import load_knesset_data, load_member_names

# Loads a dict containing mapping between knesset member id and the member name
member_names = load_member_names(use_data=USE_DATA)

# define flow steps for loading the source committee meetings data
# the actual loading is done later in the Flow
load_steps = (
    load_knesset_data('people/committees/meeting-attendees/datapackage.json', USE_DATA),
    filter_rows(**MEETINGS_FILTER_ROWS_KWARGS)
)

if not USE_DATA:
    # when loading from URL - enable caching which will skip loading on 2nd run
    load_steps = (cache(*load_steps, cache_path='.cache/people-committee-meeting-attendees-knesset-20'),)

loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/members/mk_individual/datapackage.json
using cache data from .cache/members-mk-individual-names
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/people/committees/meeting-attendees/datapackage.json


## Main processing functions

In [38]:
from collections import defaultdict
from dataflows import Flow

stats = defaultdict(int)
member_attended_meetings = defaultdict(int)

def to_dataframe(row):
    rows.append(row)
    
def protocols_to_dataframe(row):
    protocol_rows.append(row)

def process_meeting_protocol_part(row):
    stats['processed parts'] += 1

def process_meeting(row):
    stats['total meetings'] += 1
    if row['attended_mk_individual_ids']:
        for mk_id in row['attended_mk_individual_ids']:
            member_attended_meetings[mk_id] += 1
    parts_filename = row['parts_parsed_filename']
    if parts_filename:
        if PROCESS_PARTS_LIMIT and stats['processed parts'] < PROCESS_PARTS_LIMIT:
            steps = (load_knesset_data('committees/meeting_protocols_parts/' + parts_filename, USE_DATA),)
            if not USE_DATA and PROCESS_PARTS_CACHE:
                steps = (cache(*steps, cache_path='.cache/committee-meeting-protocol-parts/' + parts_filename),)
            steps += (process_meeting_protocol_part,)
            Flow(*steps).process()

process_steps = (to_dataframe,)

## Run the flow

In [39]:
from dataflows import Flow, dump_to_path

Flow(*load_steps, *process_steps, dump_to_path('data/committee-meeting-attendees-parts')).process()

using cache data from .cache/people-committee-meeting-attendees-knesset-20


(<datapackage.package.Package at 0x7f7e459e2e80>,
 {'count_of_rows': 10256,
  'bytes': 30129277,
  'hash': '9bb63d3b4c724c88df1416113d0fb80c',
  'dataset_name': None})

## Get committees data

In [42]:
for index, row in enumerate(rows):
    parts_filename = row['parts_parsed_filename']
    if parts_filename:
        if PROCESS_PARTS_LIMIT and stats['processed parts'] < PROCESS_PARTS_LIMIT:
            steps = (load_knesset_data('committees/meeting_protocols_parts/' + parts_filename, USE_DATA),)
            if not USE_DATA and PROCESS_PARTS_CACHE:
                steps = (cache(*steps, cache_path='.cache/committee-meeting-protocol-parts/' + parts_filename),)
            steps += (protocols_to_dataframe,)
            Flow(*steps).process()
    
    if len(protocol_rows) > 5:
        process_protocol_rows(protocol_rows)
        protocol_rows = []

        index += 1
        if (index + 1) % PROCESS_PARTS_LIMIT == 0:
            break

loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/committees/meeting_protocols_parts/files/5/6/562716.csv
using cache data from .cache/committee-meeting-protocol-parts/files/5/6/562716.csv
הכנסת העשרים

נוסח לא מתוקן

מושב ראשון

פרוטוקול מס' 1

מישיבת ועדת הכספים

יום רביעי, י"ב בניסן התשע"ה (01 באפריל 2015), שעה 14:10
ניסן סלומינסקי -> {'ייעוץ משפטי'}
אלי אלאלוף -> {'אתי בן יוסף'}
ניסן סלומינסקי -> {'עאידה תומא סלימאן'}
זאב אלקין -> {'עאידה תומא סלימאן'}
ניסן סלומינסקי -> {'זאב אלקין'}
ניסן סלומינסקי -> {'זאב אלקין'}
ניסן סלומינסקי -> {'אלי אלאלוף'}
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/committees/meeting_protocols_parts/files/5/6/563293.csv
using cache data from .cache/committee-meeting-protocol-parts/files/5/6/563293.csv
אישור השקעת המדינה בחברת דואר ישראל בע"מ בהתאם לסעיף 10 לחוק החברות הממשלתיות, התשל"ה-1975
ניסן סלומינסקי -> {'ניסן סלומינסקי'}
חמד עמאר -> {'נכחו'}
מיקי לוי -> {'מוזמנים'}
יעקב ליצמן -> {'מנהל הוועד

זאב אלקין -> {'אחמד טיבי', 'יואל חסון', 'עפר שלח', 'יעל גרמן'}
זאב אלקין -> {'אחמד טיבי', 'יואל חסון', 'עפר שלח', 'יעל גרמן'}
זאב אלקין -> {'באסל גטאס'}
זאב אלקין -> {'באסל גטאס', 'דניאל עטר'}
זאב אלקין -> {'באסל גטאס', 'דניאל עטר'}
זאב אלקין -> {'שרון גל'}
זאב אלקין -> {'דב חנין'}
זאב אלקין -> {'דב חנין'}
מיקי רוזנטל -> {'זאב אלקין'}
זאב אלקין -> {'יואל חסון'}
דב חנין -> {'קריאה'}
זאב אלקין -> {'מיכל רוזין'}
זאב אלקין -> {'דב חנין'}
קריאה -> {'דב חנין'}
היועץ המשפטי לכנסת איל ינון -> {'קריאה'}
ח זאב אלקין -> {'קריאה'}
עפר שלח -> {'היועץ המשפטי לכנסת איל ינון'}
זאב אלקין -> {'קארין אלהרר'}
זאב אלקין -> {'קריאה'}
זאב אלקין -> {'מרב מיכאלי'}
דניאל עטר -> {'זאב אלקין'}
זאב אלקין -> {'מרב מיכאלי'}
מיקי רוזנטל -> {'זאב אלקין'}
זאב אלקין -> {'דניאל עטר'}
מיכל רוזין -> {'זאב אלקין'}
זאב אלקין -> {'עפר שלח'}
עפר שלח -> {'אהוד ברק', 'בנימין נתניהו'}
עפר שלח -> {'אהוד ברק', 'בנימין נתניהו'}
עפר שלח -> {'זאב אלקין'}
זאב אלקין -> {'עפר שלח'}
יעל גרמן -> {'אהוד ברק'}
זאב אלקין -> {'באסל גטאס'}
זאב 

רביזיה בדבר קביעת ועדה לדיון בהצעת חוק יסוד: הממשלה (תיקון מס' 3 והוראת שעה לכנסת ה-20) (מ/915), לשם הכנסת לקריאה שנייה ושלישית
מרב מיכאלי -> {'מרב מיכאלי'}
מרב מיכאלי -> {'יואל חסון'}
זאב אלקין -> {'יואל חסון'}
זאב אלקין -> {'מרב מיכאלי'}
יואל חסון -> {'זאב אלקין'}
מרב מיכאלי -> {'זאב אלקין'}
יואל חסון -> {'מרב מיכאלי'}
זאב אלקין -> {'יואב קיש'}
זאב אלקין -> {'דוד ביטן'}
זאב אלקין -> {'מיקי זוהר'}
זאב אלקין -> {'נורית קורן'}
זאב אלקין -> {'יואל חסון'}
זאב אלקין -> {'אחמד טיבי'}
זאב אלקין -> {'עפר שלח'}
זאב אלקין -> {'איילת שקד', 'ינון מגל'}
זאב אלקין -> {'איילת שקד', 'ינון מגל'}
זאב אלקין -> {'יעקב מרגי'}
זאב אלקין -> {'מאיר פרוש'}
זאב אלקין -> {'אורי מקלב'}
זאב אלקין -> {'מאיר פרוש'}
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/committees/meeting_protocols_parts/files/5/6/564348.csv
using cache data from .cache/committee-meeting-protocol-parts/files/5/6/564348.csv
הצעת חוק יסוד: הממשלה (תיקון מס' 3 והוראת שעה לכנסת ה-20) (מ/915), התשע"ה2015
זאב אלקין -

חמד עמאר -> {'דב חנין'}
זאב אלקין -> {'יעקב מרגי', 'אילן גלאון', 'יואב קיש', 'איילת שקד', 'אורי מקלב', 'עפר שלח', 'זאב אלקין', 'חיים ילין', 'ינון מגל', 'נורית קורן', 'מאיר פרוש', 'מיקי זוהר', 'אחמד טיבי', 'יואל חסון', 'ענת ברקו', 'דוד ביטן', 'גקי לוי'}
זאב אלקין -> {'יעקב מרגי', 'אילן גלאון', 'יואב קיש', 'איילת שקד', 'אורי מקלב', 'עפר שלח', 'זאב אלקין', 'חיים ילין', 'ינון מגל', 'נורית קורן', 'מאיר פרוש', 'מיקי זוהר', 'אחמד טיבי', 'יואל חסון', 'ענת ברקו', 'דוד ביטן', 'גקי לוי'}
זאב אלקין -> {'יעקב מרגי', 'אילן גלאון', 'יואב קיש', 'איילת שקד', 'אורי מקלב', 'עפר שלח', 'זאב אלקין', 'חיים ילין', 'ינון מגל', 'נורית קורן', 'מאיר פרוש', 'מיקי זוהר', 'אחמד טיבי', 'יואל חסון', 'ענת ברקו', 'דוד ביטן', 'גקי לוי'}
זאב אלקין -> {'יעקב מרגי', 'אילן גלאון', 'יואב קיש', 'איילת שקד', 'אורי מקלב', 'עפר שלח', 'זאב אלקין', 'חיים ילין', 'ינון מגל', 'נורית קורן', 'מאיר פרוש', 'מיקי זוהר', 'אחמד טיבי', 'יואל חסון', 'ענת ברקו', 'דוד ביטן', 'גקי לוי'}
זאב אלקין -> {'יעקב מרגי', 'אילן גלאון', 'יואב קיש', 'איילת 

קביעת ועדה לדיון בהצעת חוק יסוד: הממשלה (תיקון מס' 3 והוראת שעה לכנסת ה-20) (מ/915), לשם הכנתה לקריאה שנייה ושלישית
זאב אלקין -> {'מיכל רוזין'}
זאב אלקין -> {'מיכל רוזין'}
יואל חסון -> {'יעקב מרגי'}
זאב אלקין -> {'יעקב מרגי'}
זאב אלקין -> {'מיקי לוי'}
זאב אלקין -> {'יואב קיש'}
זאב אלקין -> {'דוד ביטן'}
זאב אלקין -> {'מיקי זוהר'}
זאב אלקין -> {'נורית קורן'}
זאב אלקין -> {'יואל חסון'}
זאב אלקין -> {'אחמד טיבי'}
זאב אלקין -> {'עפר שלח'}
זאב אלקין -> {'איילת שקד', 'ינון מגל'}
זאב אלקין -> {'איילת שקד', 'ינון מגל'}
זאב אלקין -> {'יעקב מרגי'}
זאב אלקין -> {'מאיר פרוש'}
זאב אלקין -> {'אורי מקלב'}
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/committees/meeting_protocols_parts/files/5/6/564429.csv
using cache data from .cache/committee-meeting-protocol-parts/files/5/6/564429.csv
הרכב ועדות הכנסת
זאב אלקין -> {'זאב אלקין'}
איתן כבל -> {''}
איתן כבל -> {'נכחו'}
אחמד טיבי -> {'ייעוץ משפטי'}
מיקי רוזנטל -> {'איתן כבל'}
זאב אלקין -> {'מיכל רוזין'}
איתן כבל -> {'עפר של

...

1. דין וחשבון על החברות הממשלתיות לשנת 2014 (הפרטות חברות ממשלתיות ונבחרת הדירקטורים)

2. דיווח ועדכון על הפרטת חברת דואר ישראל בע"מ והשירותים הניתנים על ידה

3. דיווח ועדכון על הפרטת חברת התעשייה הצבאית לישראל בע"מ
משה גפני -> {'משה גפני'}
משה גפני -> {'סדר היום'}
משה גפני -> {'יצחק וקנין'}
מיקי רוזנטל -> {'ייעוץ משפטי'}
יצחק וקנין -> {'מנהל הוועדה'}
משה גפני -> {'רישום פרלמנטרי'}
משה גפני -> {'אורי יוגב'}
אראל מרגלית -> {'מכלוף מיקי זוהר'}
משה גפני -> {'אראל מרגלית'}
משה גפני -> {'אראל מרגלית'}
מיקי לוי -> {'אלי כהן'}
משה גפני -> {'אורי מקלב'}
משה גפני -> {'מיקי לוי'}
אורי מקלב -> {'מיקי רוזנטל'}
מכלוף מיקי זוהר -> {'אורי מקלב'}
אורי מקלב -> {'רחל עזריה'}
מיכל בירן -> {'מכלוף מיקי זוהר'}
אורי יוגב -> {'משה גפני'}
אורי מקלב -> {'משה גפני'}
אורי יוגב -> {'אורי מקלב'}
אורי יוגב -> {'יוסי יונה'}
משה גפני -> {'יוסי יונה'}
משה גפני -> {'אורי יוגב'}
משה גפני -> {'מיקי רוזנטל'}
שגית אפיק -> {'אורי יוגב'}
משה גפני -> {'מיקי רוזנטל'}
משה גפני -> {'אורי יוגב'}
אורי יוגב -> {'שגית אפיק'}
משה גפנ

using cache data from .cache/committee-meeting-protocol-parts/files/5/6/565627.csv
1. מעמד הר הזיתים כאתר לאומי

2. תחזוקת ואבטחת הגישה להר הזיתים
יואב בן צור -> {'יואב בן צור'}
בכל מקרה אחת מן השתיים -> {''}
דוד אמסלם -> {'מיקי זוהר'}
מכלוף מיקי זוהר -> {'נכחו'}
דוד אמסלם -> {'עבדאללה אבו מערוף'}
דוד אמסלם -> {'בכל מקרה אחת מן השתיים'}
ישראל אייכלר -> {'יואב בן צור'}
ישראל אייכלר -> {'מאיר נעמד'}
ישראל אייכלר -> {'יואב בן צור'}
בצלאל סמוטריץ -> {'דוד אמסלם'}
הלל הורוביץ -> {'אריה אלדד'}
דוד אמסלם -> {'שרון גל'}
שרון גל -> {'בצלאל סמוטריץ'}
יעקב יוסף גוש וואנדר -> {'דוד אמסלם'}
הלל הורוביץ -> {'דוד אמסלם'}
דוד אמסלם -> {'הלל הורוביץ'}
דוד אמסלם -> {'יעקב יוסף גוש וואנדר'}
דוד אמסלם -> {'הלל הורוביץ'}
דוד אמסלם -> {'הלל הורוביץ'}
דוד אמסלם -> {'הלל הורוביץ'}
דוד אמסלם -> {'הלל הורוביץ'}
דוד אמסלם -> {'בצלאל סמוטריץ'}
דוד אמסלם -> {'גבריאלה פיסמן'}
דוד אמסלם -> {'גבריאלה פיסמן'}
שרון גל -> {'חיים שמואלי'}
שרון גל -> {'חיים שמואלי'}
חיים שמואלי -> {'דוד אמסלם'}
דוד אמסלם -> {'בצלאל סמוטרי

using cache data from .cache/committee-meeting-protocol-parts/files/5/6/565729.csv
הוזלת מחירי הקייטנות שמפעילות הרשויות המקומיות והמתנ"סים
מלי סבח -> {'דוד אמסלם'}
מלי סבח -> {'מלי סבח'}
אתי ראובן -> {'סדר היום'}
יפעת שמר -> {'מנהלת הוועדה'}
דוד אמסלם -> {'עבדאללה אבו מערוף'}
עבדאללה אבו מערוף -> {'דוד אמסלם'}
זוהיר בהלול -> {'אתי ראובן'}
דב חנין -> {'זוהיר בהלול'}
משה גפני -> {'קריאה'}
משה גפני -> {'קריאה'}
יוסי יונה -> {'דוד אמסלם'}
משה גפני -> {'דב חנין'}
דוד אמסלם -> {'דב חנין'}
משה גפני -> {'דוד אמסלם'}
תמר זנדברג -> {'משה גפני'}
דוד אמסלם -> {'יוסי יונה'}
יוסי יונה -> {'משה גפני'}
דב חנין -> {'קריאה'}
דניאל עטר -> {'יעל גרמן'}
דוד אמסלם -> {'יוסי יונה'}
דניאל עטר -> {'דוד אמסלם'}
דוד אמסלם -> {'יעל גרמן'}
יעל גרמן -> {'שי פירון'}
יעל גרמן -> {'דוד אמסלם'}
דב קלמנוביץ -> {'יעל גרמן'}
מלכה סבח -> {'יעל גרמן'}
דב קלמנוביץ -> {'אתי ראובן'}
דוד אמסלם -> {'רחל עזריה'}
דודו ביטס -> {'דוד אמסלם'}
דוד אמסלם -> {'דודו ביטס'}
דודו ביטס -> {'דוד אמסלם'}
מיקול ניצא -> {'דודו ביטס'}
דוד אמסלם

אמיר רשף -> {'שאולי כצנלסון'}
משה גפני -> {'שרגא ברוש'}
שרגא ברוש -> {'אראל מרגלית'}
רחל עזריה -> {'אראל מרגלית'}
שרגא ברוש -> {'אראל מרגלית'}
שרגא ברוש -> {'משה גפני'}
משה גפני -> {'רחל עזריה'}
אמיר רשף -> {'אראל מרגלית'}
משה גפני -> {'שרגא ברוש'}
משה גפני -> {'שרגא ברוש'}
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/committees/meeting_protocols_parts/files/5/6/565611.csv
using cache data from .cache/committee-meeting-protocol-parts/files/5/6/565611.csv
אלימות כלפי עולים מצד מוסדות אכיפת החוק
מנואל טרכטנברג -> {'חברי הוועדה'}
רוברט אילטוב -> {'אברהם נגוסה'}
סופה לנדבר -> {'אברהם נגוסה'}
סופה לנדבר -> {'אברהם נגוסה'}
טלי פלוסקוב -> {'אברהם נגוסה'}
אבי פרנקל -> {'סופה לנדבר'}
טלי פלוסקוב -> {'סופה לנדבר'}
טלי פלוסקוב -> {'סופה לנדבר'}
טלי פלוסקוב -> {'סופה לנדבר'}
אברהם נגוסה -> {'יצחק הרצוג'}
יצחק הרצוג -> {'רוברט אילטוב'}
דב חנין -> {'רוברט אילטוב'}
סופה לנדבר -> {'רוברט אילטוב'}
טמסגן באייך -> {'אברהם נגוסה'}
סופה לנדבר -> {'קריאה'}
מוטי ברקוביץ -> {'א

1. הצעה להקמת ועדות לעניין מסויים, לפי סעיף 108 לתקנון הכנסת

2. קביעת ועדות לדיון בהצעות לסדר היום שהועברו ממליאת הכנסת

3. החלטה בדבר הגדלת מספר הסגנים ליושב-ראש הכנסת, לפי סעיף 10 לחוק הכנסת, והמלצה בדבר בחירת סגנים נוספים ליושב-ראש הכנסת.

4. הצעה לסדר-היום בנושא: "ביטול קריטריון מיצוי כושר ההשתכרות כתנאי לסבסוד מעונות יום"

5. הצעה לסדר-היום בנושא: "חרם ודה-לגיטימציה נגד ישראל בארגונים בינלאומיים"
דוד ביטן -> {'דוד ביטן'}
דוד ביטן -> {'צחי הנגבי'}
דוד ביטן -> {'שרון גל'}
דוד ביטן -> {'ישראל אייכלר'}
דוד ביטן -> {'נחמן שי', 'אורי מקלב', 'מיכאל אורן', 'אחמד טיבי', 'ענת ברקו'}
דוד ביטן -> {'נחמן שי', 'אורי מקלב', 'מיכאל אורן', 'אחמד טיבי', 'ענת ברקו'}
דוד ביטן -> {'נחמן שי', 'אורי מקלב', 'מיכאל אורן', 'אחמד טיבי', 'ענת ברקו'}
דוד ביטן -> {'נחמן שי', 'אורי מקלב', 'מיכאל אורן', 'אחמד טיבי', 'ענת ברקו'}
דוד ביטן -> {'נחמן שי', 'אורי מקלב', 'מיכאל אורן', 'אחמד טיבי', 'ענת ברקו'}
דוד ביטן -> {'יחיאל חיליק בר', 'יואל חסון'}
דוד ביטן -> {'יחיאל חיליק בר', 'יואל חסון'}
שרון גל -> {'משה גפני'

# Creating a graph of MK connections
An edge from A to B represents that A spoke to B. <br/>
The weight of such edge is the number of times A spoke to B.

In [28]:
G = nx.DiGraph()

def add_edges(speaker, member_set):
    if len(member_set) != 0:
        for member in member_set:
            print ('%s -> %s' % (speaker, member_set))
            add_edge(speaker, member)

def add_edge(speaker, member):
    speaker = speaker[::-1]
    member = member[::-1]
    if G.has_edge(speaker, member):
        G[speaker][member]['weight'] += 1
    else:
        G.add_edge(speaker,member, weight=1)
            
def clean(s):
    if s is None:
        return ''
    
    cleaned_string = remove_titles(s)
    cleaned_string = ''.join([i for i in cleaned_string if i.isalpha() or i.isspace()])
    cleaned_string = cleaned_string.strip()
    return cleaned_string
    
def remove_titles(s):
    return s.replace('היו"ר', '')

def first_non_speaker(speaker, curr_index, protocol_rows):
    i = 0
    former_speaker = speaker
    while speaker in former_speaker:
        i = i + 1
        former_speaker = clean(protocol_rows[curr_index - i]['header'])
        
    return former_speaker

def process_protocol_rows(protocol_rows):
    
    print (protocol_rows[1]['body'])

    indirect_reference = ['אתה', 'לך', 'שאתה', ' שאת ']
    self_reference = ['אני']
    
    for index, row in enumerate(protocol_rows[8:]):

        if row['header'] is None or row['body'] is None:
            print (row)
            continue

        speaker = clean(row['header'])
        spoken_text = clean(row['body'])

        direct_references = set([member for member in member_names.values() if (member in spoken_text)])
        indirect_references = set([first_non_speaker(speaker, index, protocol_rows) for word in indirect_reference if (word in spoken_text)])
        self_reference = set([speaker for word in self_reference if word in spoken_text])

        add_edges(speaker, direct_references)
        add_edges(speaker, indirect_references)
        add_edges(speaker, self_reference)

In [43]:
nx.write_gml(G, './Mk_connections_graph')